# Exploring a Single Subject

The purpose of this notebook is take all subjects and segment them assuming they are clean.

The researcher/student is not intended to make changes here.

In [1]:
import mne
import pandas as pd
import pylossless as ll
import glob
import numpy as np
import json
from pathlib import Path
mne.viz.set_browser_backend('qt')

# EEG Configuration
filter_freqs = (1.0, 30.0) # Filter data from (low_freq, high_freq)
channel_interest = [
    "E4", "E5", "E6", "E7", "E11", "E12", "E13",
    "E19", "E53", "E61", "E62", "E65", "E66", "E67",
    "E69", "E70", "E71", "E72", "E74", "E75", "E76",
    "E77", "E78", "E82", "E83", "E84", "E86", "E89",
    "E90", "E96", "E106", "E112"
]
epoch_tmin = -.2 # Baseline to stimulus onset in seconds, negative implies before stimulus
epoch_tmax = 1.0 # Duration after stimulus of epoch
reject_criteria = {'eeg': 200e-6} # 200 microvolts peak to peak
vo_conditions = ['VO21', 'VO22', 'VO23', 'VO24', 'VO25']
dv_conditions = ['DV31', 'DV32', 'DV33', 'DV34', 'DV35']
gn_conditions = ['GN41', 'GN42']
sr_conditions = ['SR11', 'SR13']
er_conditions = ['ER61', 'ER63']
all_conditions = vo_conditions + dv_conditions + gn_conditions

Using qt as 2D backend.


In [2]:
Path('derivatives/segmented').mkdir(parents=True, exist_ok=True)
Path('derivatives/segmented/vo').mkdir(parents=True, exist_ok=True)
Path('derivatives/segmented/dv').mkdir(parents=True, exist_ok=True)
Path('derivatives/segmented/sr').mkdir(parents=True, exist_ok=True)
Path('derivatives/segmented/gn').mkdir(parents=True, exist_ok=True)
Path('derivatives/segmented/er').mkdir(parents=True, exist_ok=True)

In [3]:
def clean_and_epoch(subject_id):
    project_path = '/media/tyler/PathstoneProject/PathstoneYAC'
    raw_path = f'{project_path}/sub-YAC{subject_id}/eeg/sub-YAC{subject_id}_task-AttnCtrl_eeg.edf'
    derivative_path = f'{project_path}/derivatives/pylossless/sub-YAC{subject_id}/eeg/sub-YAC{subject_id}_task-afd_eeg.edf'

    ll_state = ll.LosslessPipeline()
    ll_state = ll_state.load_ll_derivative(derivative_path)
    ll_state.raw.info['bads'] = ll_state.flags['ch'].get_flagged()
    ll_state.ica2.exclude = [index for index,comp in ll_state.flags['ic'].iterrows() if comp['ic_type'] in ['eog', 'ecg', 'muscle', 'line_noise', 'channel_noise']]

    cleaned_state = ll_state.raw.copy()
    cleaned_state.load_data()
    ll_state.ica2.apply(cleaned_state)
    cleaned_state = cleaned_state.interpolate_bads()
    cleaned_state = cleaned_state.filter(l_freq=filter_freqs[0], h_freq=filter_freqs[1])

    raw_data = mne.io.read_raw_edf(f'sub-YAC{subject_id}/eeg/sub-YAC{subject_id}_task-AttnCtrl_eeg.edf')
    base_events, event_dict = mne.events_from_annotations(raw_data)    
    cleaned_state = cleaned_state.set_eeg_reference(ref_channels="average")

    sr_raw = cleaned_state.copy()
    sr_start = base_events[np.where(base_events[2] == event_dict[sr_conditions[0]])][0][0]
    sr_end = sr_start + (240 * sr_raw.info['sfreq'])
    sr_raw = sr_raw.crop(tmin=sr_start / sr_raw.info['sfreq'], tmax=sr_end / sr_raw.info['sfreq'])
    sr_raw.save(f'derivatives/segmented/sr/yac_{subject_id}_pylqcr_raw.fif', overwrite=True)
    
    # Hard coding because clarity...
    new_event_dict = {
        'VO21': event_dict['VO21'],
        'VO22': event_dict['VO22'],
        'VO23': event_dict['VO23'],
        'VO24/combine': event_dict['VO24'],
        'VO25/combine': event_dict['VO25'],
        'DV31': event_dict['DV31'],
        'DV32': event_dict['DV32'],
        'DV33': event_dict['DV33'],
        'DV34/combine': event_dict['DV34'],
        'DV35/combine': event_dict['DV35'],
        'GN41': event_dict['GN41'],
        'GN42': event_dict['GN42'],
        'resp': event_dict['resp'],
    }
    new_events = [item for item in base_events if item[2] in event_dict.values()]
    
    epochs = mne.Epochs(
        cleaned_state,
        base_events,
        picks=channel_interest,
        event_id=new_event_dict,
        tmin=epoch_tmin,
        tmax=epoch_tmax,
        preload=True,
        reject=reject_criteria,
        event_repeated='merge',
    )

    # Trim down to only selected event types
    epochs = epochs[all_conditions]
    event_counts = {label:len(epochs[label]) for label in all_conditions}
    event_counts['subject'] = subject_id
    
    epochs[vo_conditions].save(f'derivatives/segmented/vo/yac_{subject_id}_pylqcr_epo.fif', overwrite=True)
    epochs[dv_conditions].save(f'derivatives/segmented/dv/yac_{subject_id}_pylqcr_epo.fif', overwrite=True)
    epochs[gn_conditions].save(f'derivatives/segmented/gn/yac_{subject_id}_pylqcr_epo.fif', overwrite=True)
    
    return event_counts

In [5]:
subject_ids = [item[7:] for item in sorted(glob.glob('sub-*'))]
leave_out = ['1013', '1034', '1051', '1060']

subject_report = []
for subject in subject_ids:
    if subject in leave_out:
        continue
    print('Working on subject: ', subject)
    out_str = clean_and_epoch(subject)
    subject_report.append(out_str)

df = pd.DataFrame(subject_report)
df = df.set_index('subject')
df.to_csv('event_report.csv')
df

Working on subject:  1001
Extracting EDF parameters from /media/tyler/PathstoneProject/PathstoneYAC/derivatives/pylossless/sub-YAC1001/eeg/sub-YAC1001_task-afd_eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading events from /media/tyler/PathstoneProject/PathstoneYAC/derivatives/pylossless/sub-YAC1001/eeg/sub-YAC1001_task-afd_events.tsv.
Reading channel info from /media/tyler/PathstoneProject/PathstoneYAC/derivatives/pylossless/sub-YAC1001/eeg/sub-YAC1001_task-afd_channels.tsv.
Reading electrode coords from /media/tyler/PathstoneProject/PathstoneYAC/derivatives/pylossless/sub-YAC1001/eeg/sub-YAC1001_space-CapTrak_electrodes.tsv.
Not fully anonymizing info - keeping his_id, sex, and hand info
Reading /media/tyler/PathstoneProject/PathstoneYAC/derivatives/pylossless/sub-YAC1001/eeg/sub-YAC1001_task-afd_ica1_ica.fif ...
Now restoring ICA solution ...
Ready.
Reading /media/tyler/PathstoneProject/PathstoneYAC/derivatives/pylossless/sub-YAC100

/home/tyler/Documents/pylossless/pylossless/pipeline.py:1199: RuntimeWarning: participants.tsv file not found for /media/tyler/PathstoneProject/PathstoneYAC/derivatives/pylossless/sub-YAC1001/eeg/sub-YAC1001_task-afd_eeg.edf
  self.raw = mne_bids.read_raw_bids(derivatives_path)


Reading 0 ... 823999  =      0.000 ...  1647.998 secs...
Applying ICA to Raw instance
    Transforming to ICA space (125 components)
    Zeroing out 13 ICA components
    Projecting back using 126 PCA components
Setting channel interpolation method to {'eeg': 'spline'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 94.0 mm
Computing interpolation matrix from 126 sensor positions
Interpolating 2 sensors
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 1651 samples (3.302 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s


Extracting EDF parameters from /media/tyler/PathstoneProject/PathstoneYAC/sub-YAC1001/eeg/sub-YAC1001_task-AttnCtrl_eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


...

/home/tyler/Documents/pylossless/pylossless/pipeline.py:1199: RuntimeWarning: participants.tsv file not found for /media/tyler/PathstoneProject/PathstoneYAC/derivatives/pylossless/sub-YAC1091/eeg/sub-YAC1091_task-afd_eeg.edf
  self.raw = mne_bids.read_raw_bids(derivatives_path)


Reading 0 ... 1598499  =      0.000 ...  3196.998 secs...
Applying ICA to Raw instance
    Transforming to ICA space (116 components)
    Zeroing out 18 ICA components
    Projecting back using 117 PCA components
Setting channel interpolation method to {'eeg': 'spline'}.
Interpolating bad channels.
    Automatic origin fit: head of radius 94.0 mm
Computing interpolation matrix from 117 sensor positions
Interpolating 11 sensors
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 1651 samples (3.302 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 128 out of 128 | elapsed:    7.0s finished


Extracting EDF parameters from /media/tyler/PathstoneProject/PathstoneYAC/sub-YAC1091/eeg/sub-YAC1091_task-AttnCtrl_eeg.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Used Annotations descriptions: ['2cor', '2inc', '3cor', '3inc', '4cor', '4inc', 'DV31', 'DV32', 'DV33', 'DV34', 'DV35', 'ER61', 'ER62', 'GN41', 'GN42', 'SR11', 'SR12', 'TRSP', 'VO21', 'VO22', 'VO23', 'VO24', 'VO25', 'bgin', 'boundary', 'isi+', 'resp', 'stm+', 'stm-']
EEG channel type selected for re-referencing
Applying average reference.
Applying a custom ('EEG',) reference.


IndexError: index 0 is out of bounds for axis 0 with size 0